## TP Data Cleaning (1) : Handling Missing Values

## Bienvenue aux travaux pratiques de Nettoyage des données.

Le nettoyage des données est une partie essentielle de la science des données, mais cela peut être profondément frustrant. Pourquoi certains de vos champs de texte sont-ils brouillés ? Que devez-vous faire à propos de ces valeurs manquantes ? Pourquoi vos dates ne sont-elles pas correctement formatées ? Comment pouvez-vous nettoyer rapidement des entrées de données incohérentes ? Dans ces TPs, vous apprendrez pourquoi vous rencontrez ces problèmes et, plus important encore, comment les résoudre !

Dans ces TPs, vous apprendrez comment aborder certains des problèmes de nettoyage de données les plus courants afin de pouvoir réellement analyser vos données plus rapidement. Vous travaillerez à travers cinq exercices pratiques avec des données réelles et désordonnées et répondrez à certaines de vos questions les plus fréquemment posées sur le nettoyage des données.

Dans ce Notebook, nous verrons comment traiter les valeurs manquantes.

## Jeter un premier coup d'œil aux données
La première chose que nous devrons faire est de charger les bibliothèques et l'ensemble de données que nous utiliserons.

À titre de démonstration, nous utiliserons un ensemble de données d'événements survenus lors de matchs de football. Dans le prochain exercice, vous appliquerez vos nouvelles compétences à un ensemble de données de permis de construire.

### À propos de l'ensemble de données
L'ensemble de données comporte 407 689 lignes et 100 colonnes. Chaque match est détaillé en fournissant des informations sur : la situation du match, les joueurs impliqués, les résultats et des mesures avancées telles que les valeurs de points attendus et de probabilité de victoire.

In [7]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
even_data = pd.read_csv("Building_Permits.csv" , low_memory=False)

# set seed for reproducibility
np.random.seed(0) 

ModuleNotFoundError: No module named 'pandas'

La première chose à faire lorsque vous obtenez un nouvel ensemble de données est d'y jeter un coup d'œil. Cela vous permet de vérifier s'il a été correctement lu et de vous donner une idée de ce qui se passe avec les données. Dans ce cas, voyons s'il y a des valeurs manquantes, qui seront représentées par `NaN` ou `None`.

In [ ]:
# look at the first five rows of the even_data file. 
# I can see a handful of missing data already!
even_data.head()

Oui, il semble qu'il y ait quelques valeurs manquantes.

### Combien de points de données manquants avons-nous ?

Maintenant que nous savons que nous avons effectivement des valeurs manquantes, voyons combien nous en avons dans chaque colonne.

In [ ]:
# get the number of missing data points per column
missing_values_count = even_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Cela semble être beaucoup ! Il pourrait être utile de voir quel pourcentage des valeurs de notre ensemble de données étaient manquantes pour avoir une meilleure idée de l'ampleur de ce problème :

In [ ]:
# how many total missing values do we have?
total_cells = np.product(even_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

Waouh, près d'un quart des cellules de cet ensemble de données sont vides ! Dans l'étape suivante, nous allons examiner de plus près certaines des colonnes avec des valeurs manquantes et essayer de comprendre ce qui pourrait se passer avec elles.

## Comprendre pourquoi les données sont manquantes

C'est à ce moment que nous entrons dans la partie de la science des données que j'aime appeler "intuition des données", c'est-à-dire "regarder vraiment vos données et essayer de comprendre pourquoi elles sont comme elles le sont et comment cela affectera votre analyse". Cela peut être une partie frustrante de la science des données, surtout si vous êtes nouveau dans le domaine et que vous n'avez pas beaucoup d'expérience. Pour traiter les valeurs manquantes, vous devrez utiliser votre intuition pour comprendre pourquoi la valeur est manquante. L'une des questions les plus importantes que vous pouvez vous poser pour aider à comprendre cela est la suivante :

> **Cette valeur manque-t-elle parce qu'elle n'a pas été enregistrée ou parce qu'elle n'existe pas ?**

Si une valeur manque parce qu'elle n'existe pas (comme la taille du plus vieux enfant de quelqu'un qui n'a pas d'enfants), il n'a pas de sens d'essayer de deviner ce qu'elle pourrait être. Vous voudrez probablement conserver ces valeurs en tant que `NaN`. D'autre part, si une valeur manque parce qu'elle n'a pas été enregistrée, alors vous pouvez essayer de deviner ce qu'elle aurait pu être en fonction des autres valeurs dans cette colonne et ligne. Cela s'appelle **l'imputation**, et nous allons apprendre à le faire ensuite ! :)

Prenons un exemple. En examinant le nombre de valeurs manquantes dans le dataframe `even_data`, je remarque que la colonne "TimesSec" a beaucoup de valeurs manquantes :

In [ ]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

En regardant le jeu de données `even_play2009to2016`, je peux voir que cette colonne contient des informations sur le nombre de secondes restantes dans le match lorsque la jeu a été effectué. Cela signifie que ces valeurs sont probablement manquantes parce qu'elles n'ont pas été enregistrées, plutôt que parce qu'elles n'existent pas. Il serait donc logique pour nous d'essayer de deviner ce qu'elles devraient être plutôt que de les laisser simplement en tant que NA.

D'autre part, il y a d'autres champs, comme "PenalizedTeam", qui ont également beaucoup de valeurs manquantes. Dans ce cas, cependant, le champ est manquant car s'il n'y avait pas de pénalité, il n'a pas de sens de dire *quelle* équipe a été pénalisée. Pour cette colonne, il serait plus logique de laisser vide ou d'ajouter une troisième valeur comme "neither" et l'utiliser pour remplacer les NA.

> **Astuce :** C'est un excellent endroit pour lire la documentation du jeu de données si vous ne l'avez pas déjà fait ! Si vous travaillez avec un jeu de données que vous avez obtenu d'une autre personne, vous pouvez également essayer de les contacter pour obtenir plus d'informations.

Si vous effectuez une analyse de données très méticuleuse, c'est à ce moment-là que vous examinerez chaque colonne individuellement pour déterminer la meilleure stratégie pour remplir ces valeurs manquantes. Pour le reste de ce cahier, nous couvrirons quelques techniques "rapides et sales" qui peuvent vous aider avec les valeurs manquantes, mais qui finiront probablement par supprimer certaines informations utiles ou ajouter du bruit à vos données.

# Supprimer les valeurs manquantes

Si vous êtes pressé ou n'avez pas de raison de savoir pourquoi vos valeurs sont manquantes, une option que vous avez est de simplement supprimer toutes les lignes ou colonnes contenant des valeurs manquantes. (Remarque : Je ne recommande généralement pas cette approche pour les projets importants ! Il vaut généralement la peine de prendre le temps de passer en revue vos données et d'examiner vraiment toutes les colonnes avec des valeurs manquantes une par une pour vraiment connaître votre jeu de données.)

Si vous êtes sûr de vouloir supprimer les lignes avec des valeurs manquantes, pandas dispose d'une fonction pratique, `dropna()`, pour vous aider à le faire. Essayons cela sur notre ensemble de données d'événements !

In [ ]:
# remove all the rows that contain a missing value
even_data.dropna()

il semble que cela ait supprimé toutes nos données ! 😱 C'est parce que chaque ligne de notre jeu de données avait au moins une valeur manquante. Nous pourrions avoir plus de chance en supprimant toutes les colonnes qui ont au moins une valeur manquante à la place.

In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = even_data.dropna(axis=1)
columns_with_na_dropped.head()

In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % even_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Nous avons perdu pas mal de données, mais à ce stade, nous avons réussi à supprimer tous les `NaN` de nos données.

# Remplir automatiquement les valeurs manquantes

Une autre option est d'essayer de remplir les valeurs manquantes. Pour cette prochaine partie, je vais obtenir une petite sous-section des données de la even afin qu'elles s'impriment correctement.

In [ ]:
# get a small subset of the even dataset
subset_even_data = even_data.loc[:, 'EPA':'Season'].head()
subset_even_data

Nous pouvons utiliser la fonction fillna() de Pandas pour remplir les valeurs manquantes dans un dataframe. Une option que nous avons est de spécifier par quoi nous voulons remplacer les valeurs NaN. Ici, je dis que je voudrais remplacer toutes les valeurs NaN par 0.

In [ ]:
# replace all NA's with 0
subset_even_data.fillna(0)

Je pourrais aussi être un peu plus astucieux et remplacer les valeurs manquantes par la valeur qui vient immédiatement après dans la même colonne. (Cela a beaucoup de sens pour les ensembles de données où les observations ont un certain ordre logique.)

In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the remaining na's with 0
subset_even_data.fillna(method='bfill', axis=0).fillna(0)

# Exercice

Écrivez votre propre code pour gérer les valeurs manquantes dans un ensemble de données de permis de construire.


1) Jetez un premier coup d'œil aux données
Exécutez la cellule de code suivante pour charger les bibliothèques et l'ensemble de données que vous utiliserez pour terminer l'exercice.

In [ ]:
# modules we'll use


# read in all our data


# set seed for reproducibility

Affichez les cinq premières lignes du DataFrame sf_permits.

Le jeu de données contient-il des valeurs manquantes ? 

2) Combien de points de données manquants avons-nous ?
Quel pourcentage des valeurs dans l'ensemble de données sont manquantes ? Votre réponse doit être un nombre entre 0 et 100. (Si 1/4 des valeurs dans l'ensemble de données sont manquantes, la réponse est 25.)

In [ ]:
percent_missing =

3) Comprenez pourquoi les données sont manquantes
Examinez les colonnes "Street Number Suffix" et "Zipcode" de l'ensemble de données des permis de construire. Les deux contiennent des valeurs manquantes.

Lesquelles, le cas échéant, manquent parce qu'elles n'existent pas ?
Lesquelles, le cas échéant, manquent parce qu'elles n'ont pas été enregistrées ?

4. Supprimer les valeurs manquantes : lignes
Si vous supprimez toutes les lignes de sf_permits contenant des valeurs manquantes, combien de lignes restent ?


5) Supprimer les valeurs manquantes : colonnes
Maintenant, essayez de supprimer toutes les colonnes avec des valeurs vides.

Créez un nouveau DataFrame appelé sf_permits_with_na_dropped qui a toutes les colonnes avec des valeurs vides supprimées.
Combien de colonnes ont été supprimées du DataFrame sf_permits original ? Utilisez ce nombre pour définir la valeur de la variable dropped_columns ci-dessous.

In [ ]:
sf_permits_with_na_dropped =

dropped_columns =

6) Remplir automatiquement les valeurs manquantes
Essayez de remplacer tous les NaN dans les données sf_permits par celui qui vient immédiatement après, puis remplacez tous les NaN restants par 0. Définissez le résultat sur un nouveau DataFrame sf_permits_with_na_imputed.

In [ ]:
sf_permits_with_na_imputed =